In [ ]:
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('Agg')
%matplotlib inline
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    

import statsmodels.api as sm
import copy
import os
import shutil

%matplotlib inline
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt


outcomes  = [f"z_global.{second_visit}", f"z_change_{memory_experiment_full_name}.0.{second_visit}",  f"z_change_snap_game_true_pos_rt_avrg.0.{second_visit}",
             f"median_z_global.{second_visit}", f"median_z_change_{memory_experiment_full_name}.0.{second_visit}",  f"median_z_change_snap_game_true_pos_rt_avrg.0.{second_visit}"]

In [ ]:
second_visit = 2
predictors = ["gender.0", f"log_time_difference.0.{second_visit}", 
              "Diabetes_2.0",  "Dyslipidemia.0", "Depression.0", "Hypertension.0", 
  "age_years.0", "education_level.0", "marital_status.0", "pp_smoke_catgeory.0", "log_MET_score.0", "waist_cm.0",  "log_bmi_kg_m2.0", 
  "Syst_bp.0", "Diast_bp.0", "log_ldl_conv.0", "log_hdl_conv.0", "log_Triglycerides_conv.0", "log_HbA1c_conv.0",  "log_Pl_glucose_conv.0",   "log_Albumin.0", 
  "log_Leukocyte_count.0", "log_C_reactive_protein.0",
   "log_oily_fish_gpday_bradbury.0", "log_white_fish_gpday_bradbury.0", 
   "log_red_meat_bradbury_gpd.0", "log_poultry_gpday_bradbury.0",
   "log_processed_meat_gpday_bradbury.0", "log_veg_gpday_bradbury.0",
   "log_fruit_gpday_bradbury.0",
    "log_cereals_gpday_bradbury.0", "log_bread_gpday_bradbury.0", "log_cheese_gpday_bradbury.0",
    "log_milk_gpday_bradbury.0", "log_tea_gpday_bradbury.0",
    "log_red_wine_gpd.0", "log_white_wine_gpd.0", "log_fortified_gpd.0", "log_beer_cider_gpd.0", "log_spirits_gpd.0",
   "aspirin.0",  "anxiety_tr.0", "pain_tr.0", "TAZD_Thiazide.0",  
   "loop_diuretics.0",  "potassium_diuretics.0", "beta_blockers.0", "calcium_antagonists.0", 
   "ARA_II_Antagonists_of_angiotensin_II_receptors.0", "IECA_Angiotensin_converting_enzyme_inhibitors.0",
   "Other_Hypotensive.0","hypochol_statins.0", "hypochol_others.0", "insulin.0", "sulfonylurea.0", "thiazolidinediones.0", "non_sulfonylurea_insulin_secretagogues.0", "metformin_category.0", "vitamins_minerals.0"]

n_samples = None

memory_experiment = "nm"



lasso_epsilon = 0.00001
p_val = 0.05
age_limit = 0
gender = 0
iqr_coefficient = 1.5 # if None then no standrat removal of outliers


HSCD = 'HC1' # if none than the model builder will be heteroscedascisity unaware
remove_na = True


working_dir =f"/projects/prime/ukbb/results_2024/{memory_experiment}_sg_0_{second_visit}_19_06_all_learning/"
# help_model_file_second_run = f"{working_dir}/non_na_median_z_global.2_HSCD_HC1_firs_run_lasso_0.01/final_model.csv"
# help_model_file_second_run = None
# help_model_file_second_run = f"{working_dir}/non_na_z_global.2_HSCD_HC1_firs_run_lasso_0.01/final_model.csv"
# help_model_file_second_run = f"{working_dir}/imputed_median_z_global.2_HSCD_HC1_firs_run_lasso_0.01/final_model.csv"
# help_model_file_second_run = f"{working_dir}/imputed_z_global.2_HSCD_HC1_firs_run_lasso_0.01/final_model.csv"
# help_model_file_second_run = f"{working_dir}/imputed_z_change_snap_game_true_pos_rt_avrg.0.2_HSCD_HC1_firs_run_lasso_0.01/final_model.csv"

input_file = f"{working_dir}/data_with_na.csv"
outcome = f"z_global_change.0.{second_visit}" # z_global_change.0.2
# outcome = f"z_change_snap_game_true_pos_rt_avrg.0.{second_visit}"
# outcome = f"z_change_log_pairs_matching_sum_incorrect.0.{second_visit}"

medications =["aspirin.0",  "anxiety_tr.0", "pain_tr.0", "TAZD_Thiazide.0", "loop_diuretics.0",  "potassium_diuretics.0", "beta_blockers.0", "calcium_antagonists.0", 
   "ARA_II_Antagonists_of_angiotensin_II_receptors.0", "IECA_Angiotensin_converting_enzyme_inhibitors.0",
   "Other_Hypotensive.0","hypochol_statins.0", "hypochol_others.0", "insulin.0", "sulfonylurea.0", "thiazolidinediones.0", "non_sulfonylurea_insulin_secretagogues.0", "metformin_category.0", "vitamins_minerals.0"]

df = pd.read_table(input_file, nrows=n_samples, sep=',')

if "imputed" in input_file:
  output_dir = f"{working_dir}/imputed_{outcome}"
else:
  df = df.dropna(axis="rows")
  print(f"cleaned check up db has {len(df)} rows")
  output_dir = f"{working_dir}/non_na_{outcome}"
  


print(df.head())

if HSCD is not None:
  output_dir = f"{output_dir}_HSCD_{HSCD}"

'''if help_model_file_second_run is None:
  output_dir = f"{output_dir}_firs_run_lasso_{lasso_epsilon}"
else:
  output_dir = f"{output_dir}_second_run"'''
  
if gender is None:
  output_dir = f"{output_dir}/"
else:
  output_dir = f"{output_dir}_gender_{gender}"
  df = df[df['gender.0'].eq(gender)]
  df = df.drop(columns=['gender.0'])
  predictors.remove("gender.0")
  
# we need to check for each medications variables if  not all are zeros
for medic in medications:
  if medic in predictors:
    if df[medic].eq(1).sum() == 0:
      df = df.drop(columns=[medic])
      predictors.remove(medic)
      print(f"dropped column because all zeroes: {medic}")

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

print(output_dir)

In [ ]:
if memory_experiment == "nm":
    memory_experiment_full_name = "num_memory_max_digits_remembered_correctly"
else:
    memory_experiment_full_name = "log_pairs_matching_sum_incorrect"

In [ ]:
'''predictors = ["gender.0", f"log_time_difference.0.{second_visit}","age_years.0", "education_level.0", 
              "learning_effect_prior.2",  "Diabetes_2.0",  "Dyslipidemia.0", "Depression.0", "Hypertension.0",
              "Syst_bp.0", "log_hdl_conv.0", "log_Triglycerides_conv.0", "log_HbA1c_conv.0",  
              "log_C_reactive_protein.0"]'''
'''"log_oily_fish_gpday_bradbury.0", "log_white_fish_gpday_bradbury.0", 
   "log_red_meat_bradbury_gpd.0", "log_poultry_gpday_bradbury.0",
   "log_processed_meat_gpday_bradbury.0", "log_veg_gpday_bradbury.0",
   "log_fruit_gpday_bradbury.0",
    "log_cereals_gpday_bradbury.0", "log_bread_gpday_bradbury.0", "log_cheese_gpday_bradbury.0",
    "log_milk_gpday_bradbury.0", "log_tea_gpday_bradbury.0",
    "quadr_red_wine_gpd.0", "quadr_white_wine_gpd.0", "quadr_fortified_gpd.0", "quadr_beer_cider_gpd.0", "quadr_spirits_gpd.0"]'''

In [ ]:
df = df[predictors+[outcome]]
if remove_na:
    df = df.dropna(axis="rows")
    print(f"cleaned check up db has {len(df)} rows")

In [ ]:
simple_predictors =["age_years.0", "gender.0", "education_level.0", f"log_time_difference.0.{second_visit}"]
if (gender is not None) and ("gender.0") in simple_predictors:
    simple_predictors.remove("gender.0") 

In [ ]:
global_predictors = predictors
for predictor in global_predictors:
    if predictor not in df.columns:
        print(f"predictor column  is missing {predictor}")

In [ ]:
def model_builder(df_source, predictors, outcome_var, result_dir, hcd):
    
    if not os.path.exists(result_dir):
    # The directory does not exist, create it
        os.makedirs(result_dir)
        
    y = df_source[outcome_var]
    print(f"outcome: {y}")
    
    X = df_source[predictors]
        
  
    # getting basic statistics for all predictors and standartising on the flow
   
    df_mean_std = pd.DataFrame(columns = ["predictor", "mean", "standard_deviation", "min", "max"] )
    
    predictors = X.columns 
    for i in  range(len(predictors)):  
        clmn = predictors[i]
        mean_ = X[clmn].mean()
        std_ = X[clmn].std()
        df_mean_std.loc[i] = [clmn, mean_, std_, X[clmn].min(), X[clmn].max()]
        X[clmn] = (X[clmn] - mean_)/std_
        
   
    X_with_intercept = sm.add_constant(X) 
    if hcd is not None:
        model_1= sm.OLS(y, X_with_intercept).fit(cov_type=hcd) 
    else:
        model_1= sm.OLS(y, X_with_intercept).fit() 
    
    
    with open(f"{result_dir}/ols_model_summary.txt", 'w') as model_1_file:
    # Convert the summary to a string and write it to the file
        model_1_file.write(model_1.summary().as_text())
    
    coefficients = model_1.params
    p_values = model_1.pvalues
    df_results = pd.DataFrame({'Coefficient': coefficients, 'P_value': p_values})
    
    df_results = df_results.reset_index()
    df_results = df_results.rename(columns={df_results.columns[0]: 'predictor'})
    df_results = pd.merge(df_results, df_mean_std, on = "predictor", how = 'outer')
    
    df_results.to_csv(f"{result_dir}/final_ols_model.csv", sep=',')
  
    return df_results, model_1

In [ ]:
def lasso_model_builder(df_source, predictors, outcome_var, result_dir, hcd):
    
    if not os.path.exists(result_dir):
    # The directory does not exist, create it
        os.makedirs(result_dir)
        
    y = df_source[outcome_var]
    print(f"outcome: {y}")
    
    X = df_source[predictors]
   
    
    ############ LASSO ###### 
    
    scaler = StandardScaler()
    # Initialize LassoCV with 100-fold cross-validation
    lasso_cv = LassoCV(cv=100, random_state=42, max_iter=10000)
    pipeline = make_pipeline(scaler, lasso_cv)
    pipeline.fit(X, y)
        
    lasso_cv_fitted = pipeline.named_steps['lassocv']
    
    

    summary_text = f"Best alpha (regularization strength): {lasso_cv_fitted.alpha_}\n"
    summary_text += f"Intercept {lasso_cv_fitted.intercept_}\nCoefficients:\n"
    summary_text += f"{lasso_cv_fitted.coef_}\n"
    summary_text += f"MSE path {lasso_cv_fitted.mse_path_}\n"
        
    r2_score = pipeline.score(X, y)
    summary_text += f"R^2 score: {r2_score}"
         
    print(summary_text)
        # Saving to a file
    with open(f"{result_dir}/lasso_summary.txt", 'w') as lasso_file:
        lasso_file.write(summary_text)
    
    # preparing the list of predictors with non zero coefficients (large enough)
    if abs(lasso_cv_fitted.intercept_) >=  lasso_epsilon:
        selected_features = ["const"]  
    else:  
        selected_features = []
        
    for i in range(len(predictors)):
        if abs(lasso_cv_fitted.coef_[i])>= lasso_epsilon:
            selected_features = selected_features + [predictors[i]]
   
    
    # getting coefficients and basic statistics for all predictors, incl const
   
    df_mean_std = pd.DataFrame(columns = ["predictor", "Coefficient", "mean", "standard_deviation", "min", "max"] )
    df_mean_std.loc[0] = ["const", lasso_cv_fitted.intercept_, lasso_cv_fitted.intercept_, 0, lasso_cv_fitted.intercept_, lasso_cv_fitted.intercept_]
    X_scaled = df_source[predictors]
    for i in  range(len(predictors)):  
        clmn = predictors[i]
        print(clmn)
        print(f"how many na we have prior standartisation: {X_scaled[clmn].isna().sum()}")
        mean_ = X_scaled[clmn].mean()
        std_ = X_scaled[clmn].std()
        df_mean_std.loc[i+1] = [clmn, lasso_cv_fitted.coef_[i], mean_, std_, X_scaled[clmn].min(), X_scaled[clmn].max()]
        X_scaled[clmn] =  (X_scaled[clmn] - mean_)/std_
        print(f"how many na we have after standartisation: {X_scaled[clmn].isna().sum()}")
        
    ## getting p-values for all predictors via ols model
   
    X_scaled_with_intercept = sm.add_constant(X_scaled) 
    if hcd is not None:
        model_1 = sm.OLS(y, X_scaled_with_intercept).fit(cov_type=hcd) 
    else:
        model_1 = sm.OLS(y, X_scaled_with_intercept).fit() 
    
    
    with open(f"{result_dir}/ols_model_summary.txt", 'w') as model_1_file:
    # Convert the summary to a string and write it to the file
        model_1_file.write(model_1.summary().as_text())
    
    coefficients = model_1.params
    p_values = model_1.pvalues
    df_results = pd.DataFrame({'OLS_Coefficient': coefficients, 'P_value': p_values})
    
    df_results = df_results.reset_index()
    df_results = df_results.rename(columns={df_results.columns[0]: 'predictor'})
    df_results = pd.merge(df_results, df_mean_std, on = "predictor", how = 'outer')
    
    df_results.to_csv(f"{result_dir}/lasso_eval_model.csv", sep=',')
    
    if 'const' in selected_features:
        selected_features.remove('const')
        
    X_selected = X_scaled[selected_features]
    X_selected_with_intercept = sm.add_constant(X_selected) 
    if hcd is not None:
        model_2= sm.OLS(y, X_selected_with_intercept).fit(cov_type=hcd) 
    else:
        model_2= sm.OLS(y, X_selected_with_intercept).fit() 
        
    with open(f"{result_dir}/final_model_summary.txt", 'w') as model_2_file:
    # Convert the summary to a string and write it to the file
        model_2_file.write(model_2.summary().as_text())
        
    print(model_2.summary())
    coefficients_2 = model_2.params
    p_values_2 = model_2.pvalues
    df_selected_results = pd.DataFrame({'OLS_Coefficient': coefficients_2, 'P_value': p_values_2})
    
    df_selected_results = df_selected_results.reset_index()
    df_selected_results = df_selected_results.rename(columns={df_selected_results.columns[0]: 'predictor'})
    df_selected_results = pd.merge(df_selected_results, df_mean_std, on = "predictor", how = 'outer')
    
    df_selected_results = df_selected_results[df_selected_results['predictor'].isin(selected_features + ['const'])]
    df_selected_results.to_csv(f"{result_dir}/final_model_with_lasso.csv", sep=',')
    
    return df_selected_results, model_2

In [ ]:
def show_dependencies(df_, model_df, predictors_to_show, dir):
    
    if model_df is None:
        return
    
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)
    
    print("we are building graphs")
    
    for predictor in predictors_to_show:
        
        
        print(predictor)
       
        y = np.zeros(df_.shape[0])
        # 
        # aggregate all the terms containing pedictor 
        for feat in model_df['predictor']:
            if not predictor in feat:
                continue
            if not feat in df_.columns:
                continue
            print(feat)
            
            # prepare the feature
            match feat:
                case _ if feat == predictor:
                    fun_pred = df_[predictor]
                case _ if feat == f"log_{predictor}":
                    fun_pred = np.log1p(df_[predictor])
                case _ if feat == f"quadr_{predictor}":
                    fun_pred = np.square(df_[predictor])
                case _ if feat == f"quadr_log_{predictor}":
                    fun_pred = np.square(np.log1p(df_[predictor]))
                case _:
                    fun_pred = None
                    print(f"error, non existing predictor {feat}")
            # standartise the feature
            fun_pred = (fun_pred - fun_pred.mean())/fun_pred.std()
            
            
            
            # find the coefficient
            row_feat = model_df[model_df['predictor'] == feat].iloc[0]
            coeff =  row_feat["Coefficient"]
            
            y = y + coeff *fun_pred
        
        # x_pred = (df_[predictor]    - df_[predictor].mean()) /  df_[predictor].std()
        # x = np.linspace(min(x_pred), max(x_pred), 100)  
          
        are_all_zeros = np.all(y == 0)        
        if not are_all_zeros:
            x = df_[predictor]
        
            plt.scatter(x, y)
            plt.xlabel(predictor)
            plt.ylabel('outcome')
            plt.legend()
            plt.grid(True)

            # Save the plot as a PNG file
            plt.savefig(f"{dir}/{predictor}.png", format='png', dpi=300)
            plt.show()

In [ ]:
if lasso_epsilon is not None :
        
    model_table, model = lasso_model_builder(df, global_predictors, outcome, f"{output_dir}",  HSCD)
    model_table_simple, model_simple = model_builder(df, simple_predictors, outcome, f"{output_dir}",  HSCD)
            
else:
    print(simple_predictors)
    X = df[global_predictors]
    y = df[outcome]
    assert(len(X) == len(y))
    
    duplicate_indices_X = X.index.duplicated().any()
    duplicate_indices_y = y.index.duplicated().any()

    print(f"Does X have duplicate indices? {duplicate_indices_X}")
    print(f"Does y have duplicate indices? {duplicate_indices_y}")
    
    indices_aligned = X.index.equals(y.index)

    print(f"Are the indices of X and y aligned? {indices_aligned}") 
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=1)
    
    assert(len(X_train) == len(y_train))
    assert(len(X_val) == len(y_val))
    
    '''
    X_train, X_v0, y_train, y_v0 = train_test_split(X, y, test_size=0.4, random_state=1)
    
    assert(len(X_train) == len(y_train))
    assert(len(X_v0) == len(y_v0))
    
    
    X_val, X_test, y_val, y_test = train_test_split(X_v0, y_v0, test_size=0.5, random_state=1)
    assert(len(X_val) == len(y_val))
    assert(len(X_test) == len(y_test))
    '''
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # main model    
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)
    # show_dependencies(df, model_table, predictors_to_plot, f"{output_dir}/figures")
  
    
    y_pred_val = model.predict(X_val_scaled)
   
    mae = mean_absolute_error(y_val, y_pred_val)
    mse = mean_squared_error(y_val, y_pred_val)
    rmse = mean_squared_error(y_val, y_pred_val, squared=False)  # RMSE is the square root of MSE
    r2 = r2_score(y_val, y_pred_val)
    
    validation_summary = f"vaidation mean_absolute_error  { mae}\n"
    validation_summary += f"vaidationroot mean_squared_error  {rmse}\n"
    validation_summary += f"vaidation R-squared (the coefficient of determination) {r2}\n"
    
    
    # simple model      
    X_train_simple = X_train[simple_predictors]
    X_val_simple = X_val[simple_predictors]
    scaler_simple = StandardScaler()
    X_train_simple_scaled = scaler_simple.fit_transform(X_train_simple)
    X_val_simple_scaled = scaler_simple.transform(X_val_simple)
    
    model_simple = LinearRegression()
    model_simple.fit(X_train_simple_scaled, y_train)
       
    
    y_simple_pred_val = model_simple.predict(X_val_simple_scaled)
    mae_simple = mean_absolute_error(y_val, y_simple_pred_val)
    mse_simple = mean_squared_error(y_val, y_simple_pred_val)
    rmse_simple = mean_squared_error(y_val, y_simple_pred_val, squared=False)  # RMSE is the square root of MSE
    r2_simple = r2_score(y_val, y_simple_pred_val)
    
    validation_summary += f"vaidation simple model mean_absolute_error  { mae_simple}\n"
    validation_summary += f"vaidation simple model root mean_squared_error  {rmse_simple}\n"
    validation_summary += f"vaidation simple model R-squared (the coefficient of determination) {r2_simple}\n"
    
    print(validation_summary)
    with open(f"{output_dir}/validation_summary.txt", 'w') as lasso_file:
        lasso_file.write(validation_summary)
    
            
            

In [ ]:
print(len(df))